<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/zong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Please comment your code
#[lab5]
import torch
from random import shuffle
import pprint
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
import re

from nltk.tokenize import word_tokenize
from collections import Counter
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Separate data and labels into two lists. As it is the given data set are in the form of data + labels, so to separate it
train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]
test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))]         #neg pos


# [lab3]random training set To make the model trained better, because the training set given in front of the front is separated before and after 000111, the front is all negative after all positive
# It seems that it is okay not to add it, but the accuracy is improved by adding it. Before adding is to adjust the training set according to the batch_size, otherwise it will appear when training the model, the accuracy of 0.5/1 constant situation. Although the loss is changing, the classification situation remains the same.
zipped = zip(train_data,train_label)                          #The zip function can only be used once. If you print(list(zipped)) directly, zipped will be empty afterwards, because the iterator has already been used once; if you want to reuse the result, you need to save it to a list.
Zipp = list(zipped) 
shuffle(Zipp)
train_data = [context[0] for context in Zipp]
train_label = [context[1] for context in Zipp]


def preprocessing(data_text):
  text = [s.lower() for s in data_text]                       #do preprocessing to lowercase [lab5]. Because it is easier to match elements in the dictionary if they are lowercase. It will not be the case that some words are lowercase and some are uppercase, which makes the data set more neat and makes it easier to delete words for subsequent processing.
  '''
  def remove_punctuation_re(x):                   #Delete the punctuation. At the very beginning I first used the keep expressions method for deleting punctuation. In the quotes below, if you want to try it, comment out the delete all punctuation and turn on the comment symbol for the remove method below.
    x1 = re.sub(r'[^\w\s]','',x)
    x2 = re.sub(r'\d','',x1)
    return x2

  text_re = [remove_punctuation_re(s) for s in text]
  '''
  tknzr = TweetTokenizer()
  text_t=[]                                    #完全每句话的分词的集合。train_t[0][0]每个单词
  for s in text:                        #text ==text_re
    text = tknzr.tokenize(s)              #每句话分别作了分词 text_train[0]第一句话的分词
    text_t.append(text)  
    
                                           #'''表情的保留方式  因为字典中没表情，所以就不用表情了
  def remove(x):
    t = []
    for i in range(len(x)):
      t_sub = []                            #是直接用空列表代替了
      for j in range(len(x[i])):
        if len(x[i][j])==0:
          continue
        if x[i][j] == " ":
          continue 
        else:
          x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
          if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
            x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
          if len(x[i][j]) == 0:
            continue 
          else:
            t_sub.append(x[i][j]) 
      t.append(t_sub) 
    return t

  new_text = remove(text_t)




  stop_words = sw.words()
  sww = sw.words()
  text_stop=[]    #8000
  for tokens in new_text:             #new_text == text_t
      filtered_sentence = [w for w in tokens if not w in stop_words]
      text_stop.append(filtered_sentence)
  return text_stop

pre_train= preprocessing(train_data)  #对应后面，后面还得改一下 train_stem ;test_stem
pre_test =preprocessing(test_data)

In [ ]:
# Please comment your code
#[lab5]
import torch
from random import shuffle
import pprint
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
import re

from nltk.tokenize import word_tokenize
from collections import Counter
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Separate data and labels into two lists. As it is the given data set are in the form of data + labels, so to separate it
train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]
test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))]         #neg pos


# [lab3]random training set To make the model trained better, because the training set given in front of the front is separated before and after 000111, the front is all negative after all positive
# It seems that it is okay not to add it, but the accuracy is improved by adding it. Before adding is to adjust the training set according to the batch_size, otherwise it will appear when training the model, the accuracy of 0.5/1 constant situation. Although the loss is changing, the classification situation remains the same.
zipped = zip(train_data,train_label)                          #The zip function can only be used once. If you print(list(zipped)) directly, zipped will be empty afterwards, because the iterator has already been used once; if you want to reuse the result, you need to save it to a list.
Zipp = list(zipped) 
shuffle(Zipp)
train_data = [context[0] for context in Zipp]
train_label = [context[1] for context in Zipp]


def preprocessing(data_text):
  text = [s.lower() for s in data_text]                       #do preprocessing to lowercase [lab5]. Because it is easier to match elements in the dictionary if they are lowercase. It will not be the case that some words are lowercase and some are uppercase, which makes the data set more neat and makes it easier to delete words for subsequent processing.

  def remove_punctuation_re(x):                   #Delete the punctuation. At the very beginning I first used the keep expressions method for deleting punctuation. In the quotes below, if you want to try it, comment out the delete all punctuation and turn on the comment symbol for the remove method below.
    x1 = re.sub(r'[^\w\s]','',x)
    x2 = re.sub(r'\d','',x1)
    return x2

  text_re = [remove_punctuation_re(s) for s in text]

  tknzr = TweetTokenizer()
  text_t=[]                                    #完全每句话的分词的集合。train_t[0][0]每个单词
  for s in text_re:
    text_re = tknzr.tokenize(s)              #每句话分别作了分词 text_train[0]第一句话的分词
    text_t.append(text_re)  
    
  '''表情的保留方式  因为字典中没表情，所以就不用表情了
  def remove(x):
    t = []
    for i in range(len(x)):
      t_sub = []        #是直接用空列表代替了
      for j in range(len(x[i])):
        if len(x[i][j])==0:
          continue
        if x[i][j] == " ":
          continue 
        else:
          x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
          if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
            x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
          if len(x[i][j]) == 0:
            continue 
          else:
            t_sub.append(x[i][j]) 
      t.append(t_sub) 
    return t

  new_text = remove(text_t)
  '''



  stop_words = sw.words()
  sww = sw.words()
  text_stop=[]    #8000
  for tokens in text_t:
      filtered_sentence = [w for w in tokens if not w in stop_words]
      text_stop.append(filtered_sentence)
  return text_stop

pre_train= preprocessing(train_data)  #对应后面，后面还得改一下 train_stem ;test_stem
pre_test =preprocessing(test_data)

In [ ]:
# Please comment your code
#[lab5]
import torch
from random import shuffle
import pprint
from nltk.tokenize import TweetTokenizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
import re

from nltk.tokenize import word_tokenize
from collections import Counter
import math
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Separate data and labels into two lists. As it is the given data set are in the form of data + labels, so to separate it
train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]
test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))]         #neg pos


# [lab3]random training set To make the model trained better, because the training set given in front of the front is separated before and after 000111, the front is all negative after all positive
# It seems that it is okay not to add it, but the accuracy is improved by adding it. Before adding is to adjust the training set according to the batch_size, otherwise it will appear when training the model, the accuracy of 0.5/1 constant situation. Although the loss is changing, the classification situation remains the same.
zipped = zip(train_data,train_label)                          #The zip function can only be used once. If you print(list(zipped)) directly, zipped will be empty afterwards, because the iterator has already been used once; if you want to reuse the result, you need to save it to a list.
Zipp = list(zipped) 
shuffle(Zipp)
train_data = [context[0] for context in Zipp]
train_label = [context[1] for context in Zipp]


def preprocessing(data_text):
  text = [s.lower() for s in data_text]                       #do preprocessing to lowercase [lab5]. Because it is easier to match elements in the dictionary if they are lowercase. It will not be the case that some words are lowercase and some are uppercase, which makes the data set more neat and makes it easier to delete words for subsequent processing.

  def remove_punctuation_re(x):                   #Delete the punctuation. At the very beginning I first used the keep expressions method for deleting punctuation. In the quotes below, if you want to try it, comment out the delete all punctuation and turn on the comment symbol for the remove method below.
    x1 = re.sub(r'[^\w\s]','',x)                  #
    x2 = re.sub(r'\d','',x1)
    return x2

  text_re = [remove_punctuation_re(s) for s in text]

  tknzr = TweetTokenizer()
  text_t=[]                                    #完全每句话的分词的集合。train_t[0][0]每个单词
  for s in text_re:
    text_re = tknzr.tokenize(s)              #每句话分别作了分词 text_train[0]第一句话的分词
    text_t.append(text_re)  
  '''表情的保留方式  因为字典中没表情，所以就不用表情了
  def remove(x):
    t = []
    for i in range(len(x)):
      t_sub = []        #是直接用空列表代替了
      for j in range(len(x[i])):
        if len(x[i][j])==0:
          continue
        if x[i][j] == " ":
          continue 
        else:
          x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
          if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
            x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
          if len(x[i][j]) == 0:
            continue 
          else:
            t_sub.append(x[i][j]) 
      t.append(t_sub) 
    return t

  new_text = remove(text_t)
  '''



  stop_words = sw.words()
  sww = sw.words()
  text_stop=[]    #8000
  for tokens in text_t:
      filtered_sentence = [w for w in tokens if not w in stop_words]
      text_stop.append(filtered_sentence)
  return text_stop

pre_train= preprocessing(train_data)  #对应后面，后面还得改一下 train_stem ;test_stem
pre_test =preprocessing(test_data)

In [ ]:
'''
# Please comment your code 评述你的代码 NLTK's Twitter_Sample dataset. 
# (Justify your decision) #[lab5]
'''
#[lab5]
import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#分集合;因为是给定的数据集都是数据+标签的形式，所以要把它分开

train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]

test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos

# 随机训练集 为了让模型训练的更好，因为给的训练集前面是前后分开的000111，前面都是消极后面都是积极
#看起来不加也行，加了准确率就提升了。。；没加之前是得根据batch_size 来调整训练集，要不然会出现训练模型时候，准确度0.5/1恒定的情况。 虽然loss在变，但是分类的情况却没有变。
#[lab3]
zipped = zip(train_data,train_label)   #上图说明 b 调用一次之后，再调用的 c 则为空。如果想要复用这个结果，需要保存到 list 里，即：
Zipp = list(zipped) #如果直接 print(list(zipped))，则之后 zipped为空，因为已经用完一次迭代器了；如果想要复用这个结果，需要保存到 list 里
from random import shuffle

shuffle(Zipp)
train_data = [context[0] for context in Zipp]
train_label = [context[1] for context in Zipp]


#做预处理 变小写 [lab5] 因为小写的话更容易匹配字典中的元素。就不会出现有些小写有些大写的情况，让数据集更加工整，也为了后续处理删词什么的方便。
import pprint
text_train = [s.lower() for s in train_data]
text_test = [s.lower() for s in test_data]

# 分词[lab5] 因为就是需要一个个词的来处理。通过一个或者多个词来判断整个句子的关系，那么首先要判断每个词的关系。
import pprint
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()


train_t=[]  #完全每句话的分词的集合。train_t[0][0]每个单词
for s in text_train:
  text_train = tknzr.tokenize(s)              #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
        #:(    :-(


test_t=[]     #2000
for w in text_test:
  text_test = tknzr.tokenize(w)
  test_t.append(text_test)

# 删标点，这里是保留了一部分表情。但是后来发现，因为字典中没有表情，所以可以不保留表情[lab5]
#theprincesszooz but i see what youre going at   
#yes  subjective pain may not be real
#  but that does not make it less painful
'''
还得改
clean_doc2 = re.sub(r'[^\w\s]','',corpus[1])字符
clean_doc2 = re.sub(r'\d,'',corpus[1])数字
'''
import re
def remove(x):
  t = []
  for i in range(len(x)):
    t_sub = []        #是直接用空列表代替了
    for j in range(len(x[i])):
      if len(x[i][j])==0:
        continue
      if x[i][j] == " ":
        continue 
      else:
        x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
        if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
          x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
        if len(x[i][j]) == 0:
          continue 
        else:
          t_sub.append(x[i][j]) 
    t.append(t_sub) 

  return t

text_train = remove(train_t)
text_test = remove(test_t)

##停用词  删除那些可有可无不影响句意比如a;an;the 但是又数目非常多的词。
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()
sww = sw.words()

text_train_le=[]    #8000
for tokens in text_train:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_le.append(filtered_sentence1)


text_test_le=[]   #2000
for tokens in text_test:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_le.append(filtered_sentence2)




In [ ]:

# stem
from nltk.stem.porter import *
def stemming(past_text):
  stemmer = PorterStemmer()
  train_stem = []
  for i in range(len(past_text)):
    singles = []
    for plural in past_text[i]:
      singles.append(stemmer.stem(plural))
    train_stem.append(singles)
  return train_stem

stem_train = stemming(text_train_le)  #nobodies -->nobodi
stem_test = stemming(text_test_le)


In [ ]:
#主要是为了得到n_class
unique_labels = np.unique(train_label)

lEnc = LabelEncoder()
label_train_encoded = lEnc.fit_transform(train_label)         #lEnc.transform(unique_labels)[0]+1
label_test_encoded = lEnc.fit_transform(test_label)
n_class = len(unique_labels)                                   #主要是为了得到n_class  #n_class = np.unique(unique_labels).shape[0] #2相当于

#要确定最大长度是多少 [lab4]
doc_length_list = []                                              #会得到8000句话
sum_text = sentences     #train_stem + test_stem要改的 sentences = stem_train + stem_test   
maxlength = 0
for doc in sum_text:
    doc_length_list.append(len(doc))                                  #每句话多少个分词

for index in range(len(doc_length_list)):
  if doc_length_list[index] > maxlength:
    maxlength = doc_length_list[index]
    max_index = index
#print(doc_length_list[max_index]) #21


In [ ]:
#According to 3.1, the effect of skip gram based on FastText is better.
# Use hard coding for naming.
sentences = stem_train + stem_test      # Facilitate training afterwards. Because the final model testing requires a training training set and a testing test set. This is done to facilitate some processing afterwards, such as supplementing the maximum length, etc. This way there is no need to change the overall code.
SIZE= 100
MIN_COUNT = 2                           #will led to OOV problems. Because My model ignores words with a word frequency lower than MIN_COUNT. So these words will not get the vector expression form.
WINDOW = 5
                                        #According to 3.1, it can be inferred that the size of SIZE does not have a significant effect on each word embedding model. It is fluctuating.

from gensim.models import FastText
from gensim.models import Word2Vec

ft_sg_model = FastText(sentences=sentences, size=SIZE, window=WINDOW, min_count=MIN_COUNT, workers=2, sg=1)
ft_sg_model.save('ft_sg.model')
ft_sg = FastText.load("./ft_sg.model")  

#other model (word2vec)
'''
#gensim word2vec #W2V-cbow
wv_cbow_model = Word2Vec(sentences=sentences, size=SIZE, window=WINDOW, min_count=MIN_COUNT, workers=2, sg=0)
wv_cbow_model.wv.save_word2vec_format('ted_cbow_w2v.txt', binary=False)
vectors_wv_cbow="/content/ted_cbow_w2v.txt"    #open

#W2v-skip gram
wv_sg_model = Word2Vec(sentences=sentences, size=SIZE, window=WINDOW, min_count=MIN_COUNT, workers=2, sg=1)
wv_sg_model.wv.save_word2vec_format('ted_sg_w2v.txt', binary=False)
vectors_wv_sg="/content/ted_sg_w2v.txt"   

#FastText- sg

ft_sg_model = FastText(sentences=sentences, size=SIZE, window=WINDOW, min_count=MIN_COUNT, workers=2, sg=1)
ft_sg_model.wv.save_word2vec_format('ted_sg_ft.txt', binary=False)
vectors_ft_sg="/content/ted_sg_ft.txt"    
# fasttext - cbow
ft_cbow_model = FastText(sentences=sentences, size=SIZE, window=WINDOW, min_count=MIN_COUNT, workers=2, sg=0)
ft_cbow_model.wv.save_word2vec_format('ted_cbow_ft.txt', binary=False)
vectors_ft_cbow="/content/ted_cbow_ft.txt"    
'''



In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

not_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding="ISO-8859-1").readlines()]
#处理文件 -
j = -1
for i in not_words:
  j +=1
  if i == '2-faced':
    break
neg = not_words[j:]

#处理文件+
sure_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt', 'r', encoding="ISO-8859-1").readlines()]

j = -1
for i in sure_words:
  j +=1
  if i == 'a+':
    break

pos = sure_words[j:]


nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
'''
def lem(past_text):
  lemmatizer = WordNetLemmatizer()

  text_le = []

  lemma_sentence = [lemmatizer.lemmatize(w) for w in past_text ]
  text_le.append(lemma_sentence)
  return text_le
neg_new = lem(neg)
pos_new = lem(pos)
'''


from nltk.stem.porter import *
def stem(past_text):
  stemmer = PorterStemmer()
  text_le = []
  stemmer_sentence = [stemmer.stem(plural) for plural in past_text ]
  text_le.append(stemmer_sentence)
  return text_le

neg_new = stem(neg)
pos_new = stem(pos)

In [ ]:
#只是原来的，因为neg就是单词了。不能再neg[0]
'''
大改动
'''
def match(text):
  sum_match= [] #train_match
  for i in range(len(text)):  #train_stem  取出一句话
    match = []
    for j in range(len(text[i])):   #取出一句话中的单词
      if text[i][j] in neg_new[0]:    #neg_stem  第几句话的第几个单词
        match.append(1)   #neg 1
      elif text[i][j] in pos_new[0]:  #pos_stem
        match.append(2) #pos 2
      else:
        match.append(0)
    sum_match.append(match)
  return sum_match
train_match = match(stem_train)  #这在不断的改变 train_stem  原本的话都是000
test_match = match(stem_test)
#只有stem好用
#print(train_match)
#print(test_match[:5])
#就是把我原本的要训练的话，纷纷用012 表示出来。每个词告诉他是积极消极还是不在

In [ ]:
#实现的是concat的功能。
'''
大改动
'''
def input(text,text_match):
  DF = {}

  for tokensized_doc in text: 
      for term in np.unique(tokensized_doc):
          try:
              DF[term] +=1
          except:
              DF[term] =1
  input = []
  for w in range(len(text)):                #8000 0-7999
    num = 0
    new = []
    for t in text[w]:                       #一句话 #t就是这个单词  #13
      match = []                            #目前这个就是第一句话的match
      if DF[t] >= 2:                         #0-12     这个min_count
        word_vec = ft_sg[t].tolist()
        match.append(text_match[w][num])     #w 012 []   #IndexError: list index out of range 后面的num问题
        new_embedding = word_vec + match       #sent_embedding
        new.append(new_embedding)
      num += 1
      if num >= len(text_match[w]):
        break
    input.append(new)                          # input == train_embedding
                                                #print(input[0][0]) #input 8000一句话 3一个词 101 vec+0、1

  return input

input_train = input(stem_train,train_match)         #train_stem变化
input_test = input(stem_test,test_match)           #test_stem变化
#pprint.pprint(input_train[:1])
#print('\n')
#pprint.pprint(input_test[:1])

In [ ]:
#测试用的删掉
#主要是为了得到n_class
unique_labels = np.unique(train_label)

lEnc = LabelEncoder()
label_train_encoded = lEnc.fit_transform(train_label)         #lEnc.transform(unique_labels)[0]+1
label_test_encoded = lEnc.fit_transform(test_label)
n_class = len(unique_labels)                                   #主要是为了得到n_class  #n_class = np.unique(unique_labels).shape[0] #2相当于

#要确定最大长度是多少 [lab4]
doc_length_list = []                                              #会得到8000句话
sum_text = sentences     #train_stem + test_stem要改的sentences = stem_train + stem_test   
maxlength = 0
for doc in sum_text:
    doc_length_list.append(len(doc))                                  #每句话多少个分词

for index in range(len(doc_length_list)):
  if doc_length_list[index] > maxlength:
    maxlength = doc_length_list[index]
    max_index = index
#print(doc_length_list[max_index]) #21


In [ ]:
#超参

seq_length = doc_length_list[max_index]           #（22）应该是我规定的最大序列长度512那种  maxlength 增加减少那种 #一句话最长多少个词
n_input = len(input_train[0][0])              #101 无所谓的反正都是101维度
n_class = len(unique_labels)                  #2

n_hidden = 200                                  #128
batch_size = 800                                #500 
total_epoch = 20                                #10
learning_rate = 0.001                           # 1e-4 == 0.001 le-3 == 0.01 #0.1  0.05


#把输入的规格都定成一样的， 变成句子长度是22，单词维度是101

def sent_encoded(input_text):
  sent_encoded = []
  for m in range(len(input_text)):                       # m 1-8000 #训练集  input_train  input_test
    
    encoded = [] 
    zero = []
    if len(input_text[m]) < seq_length:
      zero = [len(input_text[0][0])*[0]] * (seq_length - len(input_text[m]))#补充到22 最长 # 不单单加0； 还是需要变成101维度
      encoded = input_text[m] + zero
    else:
      encoded = input_text[m]
    sent_encoded.append(encoded)                                              #要重新添加。 句子已经全变成22了 

  sent_encoded = np.array(sent_encoded)
  return sent_encoded

sent_encoded_train = sent_encoded(input_train)
sent_encoded_test = sent_encoded(input_test)
#print(sent_encoded_train[:1]) #(8000, 22, 101)
#print(sent_encoded_test[:1])#(2000, 22, 101)


#还是Bi-LSTM准确度更高，因为LSTM处理了记忆丢失的问题。
'''
要写个LSTM比RNN好在哪里。
'''
# 模型Bi-RNN


'''
class Bi_RNN_Model(nn.Module):
    def __init__(self):
        super(Bi_RNN_Model, self).__init__()
        # set the bidirectional to True
        self.rnn = nn.RNN(n_input, n_hidden, batch_first =True, bidirectional=True) #, dropout = 0.5
        self.linear = nn.Linear(2*n_hidden,n_class) #n_class = 3 
        #https://stackoverflow.com/questions/60259836/cnn-indexerror-target-2-is-out-of-bounds 虽然是01分类但是，pytorch 要进行012 
    def forward(self, x):        
        x, h_n = self.rnn(x)
        # concat the last hidden state from two direction
        hidden_out = torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        output = self.linear(hidden_out)
        return output


'''

#LSTM-BI
class Bi_LSTM_Model(nn.Module):
    def __init__(self):
        super(Bi_LSTM_Model, self).__init__()
        #self.emb = nn.Embedding(vocab_size, emb_dim)
        # Initialize the Embedding layer with the lookup table we created 
        #self.emb.weight.data.copy_(torch.from_numpy(emb_table))
        # Optional: set requires_grad = False to make this lookup table untrainable
        #self.emb.weight.requires_grad = False

        self.lstm = nn.LSTM(n_input, n_hidden, batch_first =True, bidirectional=True)
        self.linear = nn.Linear(n_hidden*2, n_class)

    def forward(self, x):
        # Get the embeded tensor
        #x = self.emb(x)        
        # we will use the returned h_n of shape (num_layers * num_directions, batch, hidden_size): tensor containing the hidden state for t = seq_len.
        # details of the outputs from nn.LSTM can be found from: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
        lstm_out, (h_n,c_n) = self.lstm(x)
        # concat the last hidden state from two direction
        hidden_out =torch.cat((h_n[0,:,:],h_n[1,:,:]),1)
        z = self.linear(hidden_out)
        return z


'''
criterion = nn.CrossEntropyLoss()
model = Bi_RNN_Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
'''
#Bi-LSTM
# Move the model to GPU
criterion = nn.CrossEntropyLoss()
model = Bi_LSTM_Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)




In [ ]:
# Please comment your code
#[lab4][lab5]
#检查输出节点数与标签数是否一致。
#并且将输出节点数num_outputs更改为标签数。
# 因为我之前设置的class 类别是 0，1；所以这里也需要改成0，1 要不然后面会出现 target超出范围的错误。
def label(label):   
  label = np.array(label) #train_label
  #neg 1 pos2
  lab = []  #label_train
  for tag in label:
    if tag == 'neg':
      lab.append(0)
    else:
      lab.append(1)
  #print(label_train)
  #print(len(label_train))
  lab = np.array(lab)
  #test_label
  return lab

label_train = label(train_label)
label_test = label(test_label)

#要开始训练。 optimizer +loss+ backward

#处理
#sent_encoded_train = sent_encoded(input_train)
#sent_encoded_test = sent_encoded(input_test)

for epoch in range(total_epoch):
    train_loss = 0
    for ind in range(0,sent_encoded_train.shape[0],batch_size):
        input_batch = sent_encoded_train[ind:min(ind+batch_size, sent_encoded_train.shape[0])] #256,22,101
        target_batch = label_train[ind:min(ind+batch_size, sent_encoded_train.shape[0])]  #256
        input_batch_torch = torch.from_numpy(input_batch).float().to(device)  #256,22,101
        target_batch_torch = torch.from_numpy(target_batch).view(-1).to(device) #256

        model.train()
        optimizer.zero_grad()
        outputs = model(input_batch_torch)   #13,512,25 #256，22，101    #256 2 #500,22,101
        loss = criterion(outputs, target_batch_torch) #256 2    256 out,y
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    predicted = torch.argmax(outputs, 1)  # 输出值为元祖获取第二个tensor        
    acc = accuracy_score(predicted.cpu().numpy(),target_batch_torch.cpu().numpy())
    print('Epoch: %d, train loss: %.5f, train_acc:%.2f'%(epoch + 1, train_loss, acc))

print('Finished Training')
#这块是在后面评估，可删
## Prediction

model.eval()
outputs2 = model(torch.from_numpy(sent_encoded_test).float().to(device)) 
predicted2 = torch.argmax(outputs2, 1)


print(classification_report(label_test, predicted2.cpu().numpy(),digits=4))
#是改变了batch
#3.59s